# Colombians migration data analysis

## librerías, carga de datos  y primera inspección

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Cargamos el dataset desde PostgreSQL para ver si los datos necesitan mas limpieza, realizar análisis exploratorio de datos (EDA) y responder preguntas específicas sobre la migración colombiana a través de aeropuertos internacionales a través de análisis y visualizaciones en Python.

In [72]:
with open('../config.json') as f:
    config = json.load(f)
db = config["database"]

# Crear la conexión
engine = create_engine(
    f"postgresql://{db['user']}:{db['password']}@{db['host']}:{db['port']}/{db['dbname']}")

# Leer la tabla limpia desde PostgreSQL
df = pd.read_sql("SELECT * FROM registro_aeropuerto", engine, parse_dates=['fecha_de_registro'])

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   pais                   200000 non-null  object        
 1   codigo_iso_pais        200000 non-null  object        
 2   ciudad_de_residencia   200000 non-null  object        
 3   oficina_de_registro    200000 non-null  object        
 4   grupo_edad             200000 non-null  object        
 5   edad_anios             199719 non-null  float64       
 6   area_conocimiento      200000 non-null  object        
 7   sub_area_conocimiento  200000 non-null  object        
 8   nivel_academico        200000 non-null  object        
 9   estado_civil           200000 non-null  object        
 10  genero                 200000 non-null  object        
 11  etnia_de_la_persona    200000 non-null  object        
 12  estatura_cm            35658 non-null   floa

,pais,codigo_iso_pais,ciudad_de_residencia,oficina_de_registro,grupo_edad,edad_anios,area_conocimiento,sub_area_conocimiento,nivel_academico,estado_civil,genero,etnia_de_la_persona,estatura_cm,ciudad_de_nacimiento,fecha_de_registro,cantidad_de_personas,latitud,longitud
0,ESPAÑA,ESP,COMUNIDAD VALENCIANA/VALENCIA,C. VALENCIA ESP,ADULTOS,44.0,CIENCIAS DE LA SALUD,MEDICINA,NO INDICA,SOLTERO,FEMENINO,OTRO,NaN,COLOMBIA/CUNDINAMARCA/BOGOTA,2018-02-01,1,40.463667,-3.749220
1,ESTADOS UNIDOS,USA,FLORIDA/CORAL GABLES,C. MIAMI,ADULTOS,39.0,CIENCIAS DE LA SALUD,BACTERIOLOG�A,PREGRADO - PROFESIONAL,DIVORCIADO,FEMENINO,NINGUNA,155.0,COLOMBIA/NORTE DE SANTANDER/CUCUTA,2014-03-01,1,37.090240,-95.712891
2,ESPAÑA,ESP,CATALU�A/BARCELONA,C. BARCELONA,ADULTOS,29.0,CIENCIAS DE LA SALUD,ENFERMER�A,NO INDICA,SOLTERO,FEMENINO,OTRO,NaN,COLOMBIA/QUINDIO/QUIMBAYA,2022-02-01,1,40.463667,-3.749220
3,ESTADOS UNIDOS,USA,NEW JERSEY/PATERSON,C. NEWARK,ADULTOS,54.0,NINGUNA,NINGUNA,PRIMARIA,SOLTERO,FEMENINO,NINGUNA,159.0,COLOMBIA/RISARALDA/PEREIRA,2015-11-01,1,37.090240,-95.712891
4,ESPAÑA,ESP,MADRID/MADRID,C. MADRID,ADULTOS,41.0,"INGENIER�A, ARQUITECTURA Y AFINES",INGENIER�A CIVIL Y AFINES,PREGRADO - TECNOL�GICO,UNION_LIBRE,MASCULINO,OTRO,NaN,COLOMBIA/HUILA/NEIVA,2023-06-01,1,40.463667,-3.749220


## Limpieza de datos

Al ver los datos, vemos como las columnas 'edad_anios' y 'estatura_cm' tiene los valores nulos que se imputaron anteriormente. Lo que vamos a realizar es lo siguiente:

* Para la columna 'edad_anios', lo que haremos es enfocarnos en la columna 'grupo_edad' y dependiendo en que grupo este, colocaremos la media de años de ese grupo etario. Si el grupo etario aparece como 'DESCONOCIDO', dejaremos el valor de ~null~.

* Para la columna 'estatura_cm', lo que haremos es enfocarnos en las columnas 'grupo_edad' y 'genero', ya que si son del mismo grupo etario, el genero puede influir ya que un hombre puede tener diferente estatura a una mujer, por lo que dependiendo esas columnas sacaremos la media de cada valor. Si el grupo etario y el genero aparecen como 'DESCONOCIDO', dejaremos el valor de 
~ null ~.

In [89]:
df.groupby("grupo_edad")["edad_anios"].apply(lambda x: x.isnull().sum()).reset_index(name="nulls_edad_anios")

,grupo_edad,nulls_edad_anios
0,ADOLESCENCIA,0
1,ADULTOS,0
2,ADULTOS JOVENES,0
3,ADULTOS MAYORES,0
4,DESCONOCIDO,281
5,INFANCIA,0
6,PRIMERA INFANCIA,0


Vemos que todos los nulos de la columna 'edad_anios' estan en el apartado de 'DESCONOCIDO', asi que, dejaremos los datos tal cual como estan.

In [94]:
df.groupby(['grupo_edad','genero'])['estatura_cm'].apply(lambda x: x.isnull().sum()).reset_index(name='nulls_estatura_cm')

,grupo_edad,genero,nulls_estatura_cm
0,ADOLESCENCIA,DESCONOCIDO,3
1,ADOLESCENCIA,FEMENINO,2028
2,ADOLESCENCIA,MASCULINO,2089
3,ADOLESCENCIA,NO_BINARIO,1
4,ADULTOS,DESCONOCIDO,75
5,ADULTOS,FEMENINO,56829
6,ADULTOS,MASCULINO,47701
7,ADULTOS JOVENES,DESCONOCIDO,10
8,ADULTOS JOVENES,FEMENINO,11958
9,ADULTOS JOVENES,MASCULINO,11867


Con esta tabla podemos notar como hay valores nulos en varias categorias pero que se podrian clasificar de tal manera que no queden tatos nulos. Entonces, sacaremos la media para cada grupo etario y genero de la estatura e imputaremos los valores necesarios correspondientes.

In [116]:
df_filtrado = df[
    (df['grupo_edad'] != 'DESCONOCIDO') &
    (df['genero'] != 'DESCONOCIDO') &
    (df['estatura_cm'].notnull())
]

resultado = (
    df_filtrado
    .groupby(['grupo_edad', 'genero'])['estatura_cm']
    .mean()
    .reset_index(name='media_estatura_cm')
)

display(resultado)


,grupo_edad,genero,media_estatura_cm
0,ADOLESCENCIA,FEMENINO,109.796992
1,ADOLESCENCIA,MASCULINO,110.528571
2,ADULTOS,FEMENINO,160.518987
3,ADULTOS,MASCULINO,172.803710
4,ADULTOS JOVENES,FEMENINO,155.280059
5,ADULTOS JOVENES,MASCULINO,162.047143
6,ADULTOS MAYORES,FEMENINO,158.733825
7,ADULTOS MAYORES,MASCULINO,169.843198
8,INFANCIA,FEMENINO,74.903226
9,INFANCIA,MASCULINO,80.039216


## Análisis exploratorio de datos

### Variables numéricas 
Despues de cargar los datos, inspeccionamos sus datos numéricos y categóricos para entender su estructura y contenido. Esto nos ayudará a identificar patrones, tendencias y posibles anomalías en los datos de migración.

In [12]:
df.describe()

,edad_anios,estatura_cm,fecha_de_registro,cantidad_de_personas,latitud,longitud
count,199719.000000,35658.000000,200000,200000.000000,200000.000000,200000.000000
mean,44.156285,164.319479,2019-12-30 12:14:06.720000,1.005735,25.888282,-52.110284
min,0.000000,51.000000,1900-01-01 00:00:00,1.000000,-40.900557,-106.346771
25%,32.000000,158.000000,2016-09-01 00:00:00,1.000000,6.423750,-95.712891
50%,43.000000,165.000000,2020-02-01 00:00:00,1.000000,37.090240,-66.589730
75%,56.000000,171.000000,2023-06-01 00:00:00,1.000000,40.463667,-3.749220
max,99.000000,264.000000,2025-10-01 00:00:00,8.000000,64.963051,174.885971
std,16.617340,11.607863,NaN,0.089622,23.490843,49.479968
